In [61]:
import torch
from torch import nn, optim, autograd as grad
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms, datasets 
import time
import random

This code classifies 10 different classes of images

In [62]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.convL1 = nn.Conv2d(in_channels=3,out_channels=64, 
                                kernel_size=(3,3), stride= (1,1),padding=2)
        nn.init.kaiming_normal(self.convL1.weight)
        self.convL2 = nn.Conv2d(in_channels=64,out_channels=48, 
                                kernel_size=(3,3), stride= (1,1),padding=2)
        nn.init.kaiming_normal(self.convL2.weight)
        self.convL3 = nn.Conv2d(in_channels=48,out_channels=32, 
                                kernel_size=(2,2), stride= (1,1),padding=1)
        nn.init.kaiming_normal(self.convL1.weight)


        self.linL4 = nn.Linear(1568, 10, bias=True)
        nn.init.xavier_uniform_(self.linL4.weight)
        self.pool3 = nn.MaxPool2d((3,3), (3,3))
        self.pool2 = nn.MaxPool2d((2,2), (2,2))


        self.relu = nn.ReLU()  
        self.Batch_norm = nn.BatchNorm1d(num_features=32)
        self.Lrelu = nn.LeakyReLU()
        self.softplus = nn.Softplus()
        self.Softmax = nn.Softmax()

    def forward(self, x):
        self.Batch_norm(x)
        x=self.Lrelu(self.convL1(x))
        x=self.pool3(x)
        x=self.relu(self.convL2(x))
        x=self.pool2(x)
        x=self.relu(self.convL3(x))

        x=torch.flatten(x)
        x=self.Softmax(self.linL4(x))
        
        return x

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

Additional Functions

In [63]:
def ret_shape(training_set)->str:
    ret_string = ""
    for idx, data in enumerate(training_set):
            datas = data[0]
            labels = data[1]
            ret_string+="{}\n".format(datas.shape)
            ret_string+="Labels:{}\n".format(labels)
            ret_string+="Labels shape:{}\n".format(len(labels))
            ret_string+="Labels[0] shape:{}\n".format(labels[0].shape)
            break
    return ret_string


def max_index(max_tnsr):
    max_tnsr = max_tnsr.tolist()
    max_num = 0
    for num in max_tnsr:
        if max_num < num:
            max_num = num
    return max_tnsr.index(max_num)


Train Cell:

In [64]:
FILE_PATH = "model.pth"

def train_net(cnn, train_ds, dev):
    start_time = time.time
    iter = 0

    # Training loop part
    for sample in train_ds:
        
        targetY = torch.nn.functional.one_hot(
            torch.tensor(sample[1]), num_classes=10)
        targetY = torch.tensor(targetY, dtype=torch.float32, requires_grad=True)
        targetY = torch.squeeze(targetY)
        optimizer = optim.SGD(params=cnn.parameters(), lr=5e-3,
                          momentum=0.799,
                          weight_decay=0.0001)
        #optimizer = optim.Adam(params=cnn.parameters(), weight_decay=0.1
        #                      , lr=1e-4, betas=[0.09, 0.0999])
        
        pred = cnn.forward(torch.squeeze(sample[0]))
        loss_func = nn.CrossEntropyLoss()
        # ---BackPropagation---
        loss = loss_func(pred, targetY)
        loss.backward()
        optimizer.step()

        if iter%100==0:
            torch.save(cnn.cpu(), FILE_PATH)
            grad_tmp = cnn.linL4.weight.grad
            print(iter," | ",grad_tmp.sum()/len(grad_tmp))

        optimizer.zero_grad()
        iter+=1


In [65]:
if __name__ == '__main__':

    # Accessing processing unit
    if torch.cuda.is_available() : device = "cuda:0"
    else : device = "cpu"
    device = torch.device(device)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
    net=CNN().to(device)

    # Loading the dataset
    train_data = torchvision.datasets.CIFAR10(
        root= "C:/Users/orian/OneDrive/שולחן העבודה/My Coding Files"
        ,train=True, transform=transforms.Compose([transforms.ToTensor()])
        #, target_transform=transforms.Compose([transforms.ToTensor()])
    )
    
    train_data = DataLoader(train_data, shuffle=True)
    train_net(net, train_data, dev=device)


C:\Users\orian\AppData\Local\Temp\ipykernel_24940\3498821549.py:6: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.
  nn.init.kaiming_normal(self.convL1.weight)
C:\Users\orian\AppData\Local\Temp\ipykernel_24940\3498821549.py:9: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.
  nn.init.kaiming_normal(self.convL2.weight)
C:\Users\orian\AppData\Local\Temp\ipykernel_24940\3498821549.py:12: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.
  nn.init.kaiming_normal(self.convL1.weight)
C:\Users\orian\AppData\Local\Temp\ipykernel_24940\4259148847.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(sample[1]), num_classes=10)
C:\Users\orian\AppData\Local\Temp\ipykernel_24940\4259148847.py:1

0  |  tensor(-1.1772e-07)
100  |  tensor(1.6391e-08)
200  |  tensor(5.0664e-08)
300  |  tensor(-8.9407e-09)
400  |  tensor(-1.4901e-09)
500  |  tensor(2.0564e-07)
600  |  tensor(5.9605e-09)
700  |  tensor(-5.2899e-08)
800  |  tensor(1.7881e-07)
900  |  tensor(8.0466e-08)
1000  |  tensor(-7.3761e-08)
1100  |  tensor(-1.6093e-07)
1200  |  tensor(-4.9174e-08)
1300  |  tensor(-1.6391e-08)
1400  |  tensor(2.4438e-07)
1500  |  tensor(7.7486e-08)
1600  |  tensor(1.6917e-07)
1700  |  tensor(-2.8461e-07)
1800  |  tensor(-1.7285e-07)
1900  |  tensor(3.3975e-07)
2000  |  tensor(-4.0531e-07)
2100  |  tensor(4.4703e-09)
2200  |  tensor(8.3447e-08)
2300  |  tensor(1.3709e-07)
2400  |  tensor(1.4901e-08)
2500  |  tensor(2.3842e-08)
2600  |  tensor(-2.5928e-07)
2700  |  tensor(9.5367e-08)
2800  |  tensor(-4.8876e-07)
2900  |  tensor(-2.3395e-07)
3000  |  tensor(1.1027e-07)
3100  |  tensor(6.5565e-08)
3200  |  tensor(-1.4752e-07)
3300  |  tensor(4.3213e-06)
3400  |  tensor(-1.3113e-07)
3500  |  tensor(

Test Cell:

In [66]:
FILE = "model.pth"
model = torch.load(FILE)
model.eval()

test_data = torchvision.datasets.CIFAR10(
        root= "C:/Users/orian/OneDrive/שולחן העבודה/My Coding Files"
        ,train=False, transform=transforms.Compose([transforms.ToTensor()]), 
    )

image_lst = [0]*10
right_pred = 0
for i in range(len(test_data)):
    if max_index(model.forward(test_data[i][0]))==test_data[i][1]:
        right_pred+=1
    image_lst[max_index(model.forward(test_data[i][0]))]+=1
print("model's accuracy:",right_pred/100,"%")
print(image_lst)

C:\Users\orian\AppData\Local\Temp\ipykernel_24940\2039709238.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(FILE)


model's accuracy: 30.3 %
[882, 932, 0, 2478, 1800, 437, 535, 0, 793, 2143]
